In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import torch
import torchvision
from torch import optim
from torch import nn
import torch.utils.data
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from torch.autograd import Variable
from matplotlib import image as img
from torchvision import datasets, transforms, models
import torch.utils.data as data
from torch.utils.data import Dataset
import os

In [2]:
# Define the data transformations for data augmentation and normalization
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a consistent size
    transforms.ToTensor(),  # Convert images to PyTorch tensors
])

augmentation = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(40),
    transforms.Resize((224, 224)),
    transforms.RandomResizedCrop(size=(224,224)),
    transforms.ToTensor(),
])

# Set the path to your dataset
data_dir = './pellegrin_db'

# Create a dataset from the image folders
dataset = datasets.ImageFolder(data_dir, transform=transform)
augmentation_ds = datasets.ImageFolder(data_dir, transform=augmentation)
dataset = torch.utils.data.ConcatDataset([dataset, augmentation_ds])

# Define the ratio for train and test split
train_ratio = 0.8  # 80% of the data for training, 20% for testing

# Calculate the sizes of the train and test sets
num_data = len(dataset)
num_train = int(train_ratio * num_data)
num_test = num_data - num_train

# Split the dataset into train and test sets
train_set, test_set = random_split(dataset, [num_train, num_test])

# Define batch size and create data loaders for train and test sets
batch_size = 16
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

# Check the length of train and test sets
print(f"Number of training examples: {len(train_set)}")
print(f"Number of testing examples: {len(test_set)}")

# You can now use train_loader and test_loader for training and testing your binary classification model.



Number of training examples: 313
Number of testing examples: 79


In [6]:
import torch.nn as nn
import torchmetrics

class CNN(nn.Module):
    def __init__(self, in_channels, num_classes):

        """
        Building blocks of convolutional neural network.

        Parameters:
           * in_channels: Number of channels in the input image (for grayscale images, 1)
           * num_classes: Number of classes to predict. In our problem, 10 (i.e digits from  0 to 9).
        """
        super(CNN, self).__init__()


        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(128 * 28 * 28, 512)  # after 3 poolings: 224 -> 112 -> 56 -> 28
        self.fc2 = nn.Linear(512, num_classes)
        
        # Dropout
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 224 -> 112
        x = self.pool(F.relu(self.conv2(x)))  # 112 -> 56
        x = self.pool(F.relu(self.conv3(x)))  # 56 -> 28
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print( torch.cuda.is_available())
model = CNN(in_channels=3, num_classes=2).to(device)
#print(model)

True


In [8]:
def train(model, train_loader, val_loader, criterion, optimizer, epochs):
    a_train_loss = []
    a_train_accuracy = []
    a_val_loss = []
    a_val_accuracy = []
    
    for epoch in range(epochs):
        train_loss = 0
        train_corrects = 0
        total = 0 
        for batch, (inputs, labels) in enumerate(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            pred = torch.max(outputs.data, 1)[1]
            train_corrects += (pred == labels).sum().item()

        train_loss /= len(train_loader.dataset)
        train_acc = train_corrects/ len(train_loader.dataset)

        model.eval()
        val_loss = 0.0
        val_corrects = 0
        with torch.no_grad():
            for batch, (inputs, labels) in enumerate(val_loader):
                inputs = inputs.to(device) 
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                pred = torch.argmax(outputs, 1)
                val_corrects += (pred == labels).sum().item()

        val_loss /= len(val_loader.dataset)
        val_acc = val_corrects/len(val_loader.dataset)
        
        a_train_loss.append(train_loss)
        a_train_accuracy.append(train_acc)
        a_val_loss.append(val_loss)
        a_val_accuracy.append(val_acc)
        
        print(f"Epoch {epoch+1}/{epochs} - Training Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}, "
                  f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")
    return model, a_train_loss, a_train_accuracy, a_val_loss, a_val_accuracy

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
model, train_loss, train_accuracy, test_loss, test_accuracy = train(model, train_loader, test_loader, criterion, optimizer, 10)

KeyboardInterrupt: 

In [ ]:
training_stats = pd.DataFrame({
    'train_loss': train_loss,
    'train_accuracy': train_accuracy,
    'test_loss': test_loss,
    'test_accuracy': test_accuracy
})
training_stats.to_csv('./training_stats/bighorn_CNN_stats.csv', index=False)
torch.save(model,'./models/bighorn_male-female_CNN_classification.pth' )

In [10]:
model = torch.load('./models/bighorn_male-female_CNN_classification.pth', weights_only = False)

In [11]:
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

corr = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        corr += (predicted == labels).sum().item()

print('Accuracy of the model on all test Data: %d %%' % ( 100 * corr / total))

Accuracy of the model on all test Data: 81 %
